In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import datetime

# 따릉이 정류소 데이터. 806503개
station = pd.read_csv('../data/station(21.01.31).csv')

# 계산용 numpy array
station_array = np.array(station.iloc[:, 0])

# 따릉이 이용 데이터
# index : [자전거번호, start_time, 대여 대여소번호, 대여 대여소명, 대여거치대, end_time,반납대여소번호,반납대여소명, 반납거치대, using_time, distance]
use = np.load('../data/data.npz', allow_pickle = True)
use = use['arr_0']

# 트래쉬 데이터 버리기

In [4]:
# 출발, 도착 정류소의 위치 좌표가 있는지 True, False 값으로 반환
start = np.isin(use[:, 2], station_array)
end = np.isin(use[:, 6], station_array)

# 출발, 도착 정류소 모두 있는 경우 1 반환
real = start * end

# 출발, 도착 정류소 정보가 모두 있는 데이터 수. 801335개
sum_real = sum(real)

# 필요없는 데이터 index
index_f = np.where(real == 0)[0]

# 필요없는 데이터 삭제한 데이터
real_station = np.delete(use, index_f, 0)

# 데이터 저장
np.savez('./data/selected_data', real_station)

# 주말, 평일 나누기

In [ ]:
# 따릉이 데이터
use = np.load('./data/selected_data.npz', allow_pickle = True)
use = use['arr_0']

# 주말 체크 (0~4: 평일, 5~6: 주말)
def is_weekend(x):
    if x>4:
        return 0
    else:
        return 1

# 주말, 평일 데이터 나누기
weekday_list = []
weekend_list = []

for i in use:
    if is_weekend(datetime.strptime(i[1][:10], '%Y-%m-%d').weekday()) == True:
        weekday_list.append(i)
    else:
        weekend_list.append(i)

weekday_list = np.array(weekday_list)
weekend_list = np.array(weekend_list)

# 데이터 저장
np.savez('./data/weekday_data', weekday_list)
np.savez('./data/weekend_data', weekend_list)

# 출, 퇴근 데이터 평일 데이터에서 나누기

In [ ]:
# 평일 따릉이
use = np.load('./data/weekday_data.npz', allow_pickle = True)
use = use['arr_0']

# 출퇴근 시간 함수 (시간은 임의로 3시간 정함)
def gotowork(x):
    if x == 7 or x == 8 or x == 9:
        return 1
    elif x == 17 or x == 18 or x == 19:
        return 2

# 출근, 퇴근 데이터 나누기
on_list = []
off_list = []

for i in use:
    if gotowork(datetime.strptime(i[1][11:], '%H:%M').hour) == 1:
        on_list.append(i)
    elif gotowork(datetime.strptime(i[1][11:], '%H:%M').hour) == 2:
        off_list.append(i)

on_list = np.array(on_list)
off_list = np.array(off_list)

# 데이터 저장
np.savez('./data/on_data', on_list)         #94811개
np.savez('./data/off_data', off_list)       #137635개